In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Goiás - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_GO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Goiás - Produção de Cimento (t)', axis=1)
data

,Ano,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiás - Consumo de Cimento (t)
0,2008,0.750753,0.786865,0.740091,9.979485e+07,5.727076e+06,14.203138,8.592012e+07,0.726402,7.941146,...,2.868108,11.760983,25.567101,931.224932,1376.621797,23.533988,16.021792,67.029130,1.258587,2310.616000
1,2009,0.751479,0.787352,0.738646,1.005138e+08,5.757645e+06,14.197250,8.596210e+07,0.729972,7.969157,...,2.305885,9.472585,26.358065,931.509364,1404.486199,23.539793,15.895700,66.852293,1.302609,2335.717000
2,2010,0.752088,0.787273,0.737500,1.030813e+08,5.806966e+06,14.241313,8.663480e+07,0.733384,7.932921,...,1.958246,9.332485,26.324590,931.774922,1422.397943,23.407790,15.594180,66.528157,1.346857,2657.527000
3,2011,0.752581,0.786636,0.736643,1.054535e+08,5.798734e+06,14.278825,8.718791e+07,0.736630,7.921619,...,2.079088,10.985689,26.891246,932.026640,1424.732773,23.379311,15.713953,67.017525,1.343835,2901.707000
4,2012,0.752723,0.785429,0.735750,1.075784e+08,5.775231e+06,14.289292,8.762067e+07,0.739409,7.786290,...,2.183282,8.159014,28.111519,932.153867,1426.391027,23.445549,15.826075,67.022758,1.300550,3061.533000
5,2013,0.753450,0.783632,0.736229,1.092519e+08,5.758466e+06,14.261467,8.779968e+07,0.742860,8.389968,...,2.403445,7.913561,27.073883,932.317691,1424.468340,23.563897,15.775697,66.406360,1.252025,3181.778000
6,2014,0.753571,0.781434,0.735915,1.105942e+08,5.711944e+06,14.238295,8.787916e+07,0.745598,9.015756,...,2.748397,10.350442,25.841819,932.468684,1423.836782,23.878739,15.683323,65.088818,1.241766,2861.897429
7,2015,0.753286,0.778804,0.735658,1.115113e+08,5.628820e+06,14.206629,8.784849e+07,0.747481,9.398111,...,2.501529,12.454398,25.709892,932.508663,1430.027526,24.014961,15.540525,64.054609,1.245195,2517.290743
8,2016,0.756276,0.784027,0.728777,1.120977e+08,5.573901e+06,14.184179,8.781949e+07,0.758668,9.378450,...,2.073464,13.111908,26.601698,932.486431,1433.688880,23.900472,15.251199,63.441573,1.236011,2193.545829
9,2017,0.755573,0.780939,0.729212,1.123228e+08,5.555871e+06,14.177721,8.783412e+07,0.759092,8.980786,...,2.177386,9.479812,26.603489,932.515580,1426.852240,23.876158,15.213263,63.423776,1.188108,2270.346000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.750753,0.786865,0.740091,9.979485e+07,5.727076e+06,14.203138,8.592012e+07,0.726402,7.941146,5.483335e+08,...,-7723.574973,2.868108,11.760983,25.567101,931.224932,1376.621797,23.533988,16.021792,67.029130,1.258587
1,0.751479,0.787352,0.738646,1.005138e+08,5.757645e+06,14.197250,8.596210e+07,0.729972,7.969157,5.302764e+08,...,-8961.781709,2.305885,9.472585,26.358065,931.509364,1404.486199,23.539793,15.895700,66.852293,1.302609
2,0.752088,0.787273,0.737500,1.030813e+08,5.806966e+06,14.241313,8.663480e+07,0.733384,7.932921,5.046425e+08,...,-9895.682385,1.958246,9.332485,26.324590,931.774922,1422.397943,23.407790,15.594180,66.528157,1.346857
3,0.752581,0.786636,0.736643,1.054535e+08,5.798734e+06,14.278825,8.718791e+07,0.736630,7.921619,4.854622e+08,...,-9989.170714,2.079088,10.985689,26.891246,932.026640,1424.732773,23.379311,15.713953,67.017525,1.343835
4,0.752723,0.785429,0.735750,1.075784e+08,5.775231e+06,14.289292,8.762067e+07,0.739409,7.786290,4.867661e+08,...,-9429.326328,2.183282,8.159014,28.111519,932.153867,1426.391027,23.445549,15.826075,67.022758,1.300550
5,0.753450,0.783632,0.736229,1.092519e+08,5.758466e+06,14.261467,8.779968e+07,0.742860,8.389968,5.225848e+08,...,-7157.441850,2.403445,7.913561,27.073883,932.317691,1424.468340,23.563897,15.775697,66.406360,1.252025
6,0.753571,0.781434,0.735915,1.105942e+08,5.711944e+06,14.238295,8.787916e+07,0.745598,9.015756,5.676413e+08,...,-3453.461327,2.748397,10.350442,25.841819,932.468684,1423.836782,23.878739,15.683323,65.088818,1.241766
7,0.753286,0.778804,0.735658,1.115113e+08,5.628820e+06,14.206629,8.784849e+07,0.747481,9.398111,5.179392e+08,...,-1442.919387,2.501529,12.454398,25.709892,932.508663,1430.027526,24.014961,15.540525,64.054609,1.245195
8,0.756276,0.784027,0.728777,1.120977e+08,5.573901e+06,14.184179,8.781949e+07,0.758668,9.378450,4.524144e+08,...,-3128.377228,2.073464,13.111908,26.601698,932.486431,1433.688880,23.900472,15.251199,63.441573,1.236011
9,0.755573,0.780939,0.729212,1.123228e+08,5.555871e+06,14.177721,8.783412e+07,0.759092,8.980786,4.213689e+08,...,-5386.337108,2.177386,9.479812,26.603489,932.515580,1426.852240,23.876158,15.213263,63.423776,1.188108


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2335.717000
1     2657.527000
2     2901.707000
3     3061.533000
4     3181.778000
5     2861.897429
6     2517.290743
7     2193.545829
8     2270.346000
9     2282.903000
10    2367.716000
11    2438.585000
12    2782.135000
13    2807.973000
14            NaN
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.667342,0.949720,1.410033,-1.833449,0.433923,-0.325155,-1.986396,-1.569745,-1.027093,1.204752,...,-0.569731,2.026352,0.984445,-1.331393,-2.215071,-2.920210,-0.729782,1.542463,1.077765,0.191401
1,-1.194211,1.061657,1.060251,-1.672467,0.753723,-0.464589,-1.926962,-1.269707,-0.979720,0.820657,...,-0.838980,0.085558,-0.051710,-0.314120,-1.543139,-1.011487,-0.705275,1.108832,0.970139,0.769359
2,-0.797383,1.043475,0.782547,-1.097605,1.269707,0.578899,-0.974517,-0.982875,-1.041004,0.275392,...,-1.042057,-1.114490,-0.115145,-0.357173,-0.915796,0.215476,-1.262514,0.071898,0.772865,1.350269
3,-0.475496,0.897221,0.575051,-0.566452,1.183590,1.467272,-0.191390,-0.710011,-1.060119,-0.132596,...,-1.062386,-0.697344,0.633403,0.371613,-0.321146,0.375413,-1.382735,0.483799,1.070702,1.310592
4,-0.382925,0.620129,0.358866,-0.090684,0.937709,1.715155,0.421338,-0.476466,-1.288995,-0.104860,...,-0.940648,-0.337667,-0.646476,1.941026,-0.020591,0.489004,-1.103118,0.869388,1.073887,0.742316
5,0.090857,0.207420,0.474906,0.284004,0.762311,1.056209,0.674789,-0.186418,-0.268017,0.657046,...,-0.446625,0.422337,-0.757614,0.606505,0.366422,0.357299,-0.603521,0.696138,0.698737,0.105254
6,0.169632,-0.297081,0.398809,0.584563,0.275613,0.507437,0.787318,0.043747,0.790354,1.615453,...,0.358807,1.613108,0.345772,-0.978073,0.723121,0.314037,0.725548,0.378463,-0.103139,-0.029436
7,-0.016584,-0.901022,0.336552,0.789891,-0.594012,-0.242472,0.743896,0.202034,1.437016,0.558230,...,0.796000,0.760923,1.298413,-1.147746,0.817566,0.738105,1.300597,-0.112621,-0.732575,0.015582
8,1.932066,0.298219,-1.329669,0.921200,-1.168560,-0.774142,0.702846,1.142334,1.403764,-0.835563,...,0.429496,-0.716758,1.596124,-0.000780,0.765046,0.988910,0.817294,-1.107619,-1.105679,-0.104987
9,1.474402,-0.410869,-1.224492,0.971602,-1.357181,-0.927081,0.723547,1.178003,0.731211,-1.495939,...,-0.061498,-0.358019,-0.048438,0.001524,0.833907,0.520597,0.714653,-1.238081,-1.116510,-0.733882


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.048551  , -1.49823661, -1.31974416,  0.77589662,
         -1.06999983, -1.13620542,  0.46780016,  1.21002656,
          0.67235226, -1.0453709 ,  1.21859064, -0.99976018,
          0.79428371,  1.15575749, -0.52559858,  1.30260559,
         -0.55253046, -0.25130359,  1.38787176,  1.47154451,
         -1.80649616,  1.09296291,  0.94281138,  0.52836084,
          0.61258103,  0.87656257,  1.47801536,  1.71982086,
          1.28221181,  1.39086472,  1.58209844,  1.08872091,
          0.55212962, -1.81398118,  1.44113674,  1.11191094,
          1.30470835,  0.86390516, -1.67365968, -1.70738939,
         -1.55813196, -0.48498301, -0.75714639,  0.40770893,
         -0.27394734,  2.02323219,  1.2908951 ,  1.70465513,
          1.81148084,  1.39968653, -1.36191442,  1.28024593,
          1.11054795,  0.12662799,  1.40414112,  1.40468999,
          1.45643959,  1.58509336,  1.32283157,  1.30222047,
          1.55138602,  1.91801602, -0.77489784,  2.32098884,
         -1.1942541 , -1

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     2335.717000
1     2657.527000
2     2901.707000
3     3061.533000
4     3181.778000
5     2861.897429
6     2517.290743
7     2193.545829
8     2270.346000
9     2282.903000
10    2367.716000
11    2438.585000
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    2438.585
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-1.158228,-2.357097,-1.338521,0.495067,-0.996697,-1.327004,0.145995,1.141532,0.318701,1.108405,...,0.964765,-1.482779,-2.260447,0.946592,0.252129,0.101107,0.76294,-1.953922,-1.523095,-2.072592


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-1.15822784, -2.35709698, -1.33852115,  0.49506741,
         -0.99669724, -1.32700383,  0.14599457,  1.14153249,
          0.3187012 ,  1.10840491,  2.63582309, -0.18995911,
          0.72640489,  2.66833308, -0.032632  ,  0.98802435,
          0.24980494, -0.08955808,  1.85434394,  2.02706568,
         -2.21150703,  1.42938591,  0.85132668,  0.14092386,
          0.30902324,  1.00267904,  2.27970137,  1.11272593,
          0.61007902,  0.89064538,  1.42231438,  1.02650562,
          1.47127593, -1.53201473,  1.73503858,  1.66610613,
          1.0456894 ,  0.55289259, -1.81216593, -2.10342296,
         -2.55377478,  0.41940628, -1.75756445,  0.241023  ,
         -0.98659345,  1.52184916,  1.01398607,  1.49290967,
          1.40764092,  1.87930411, -2.51892072,  2.83356214,
          1.58620924, -0.02524128,  1.54231542,  1.5176119 ,
          1.7355269 ,  1.87941982,  1.54198129,  1.51276712,
          1.83795594,  2.62210693,  1.04993174,  0.96476493,
         -1.48277861, -2

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    2782.135
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [21]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [22]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [23]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3281328673, 717185974, 1060477234, 1017327728, 2766259312, 2516245837, 2748044371, 2194623005, 495793467, 4069185169, 762815371, 2216014430, 1765656405, 3635407162, 967673557, 3671347576, 3937505714, 1941772737, 133920037, 2284561447, 3929125198, 3035796487, 2764996907, 1561375330, 592851214, 3348919342, 219794190, 348130228, 2831652175, 1926411384, 2638953674, 1765813509, 3409520744, 1149429658, 4252320703, 1448975454, 3151424513, 2908323956, 3255217267, 2745406048, 2065222312, 3195788364, 3342157923, 995757404, 511848262, 2743979763, 3818038833, 2025435485, 1532665706, 1001974328, 1684428974, 4193523269, 54814928, 3435249121, 353030225, 3551460267, 178092728, 3485180594, 571834017, 4268194096, 1707452904, 1246047120, 150597330, 2100324476, 1740949807, 1897398143, 16728048, 419372957, 2137349011, 214472767, 847292255, 1545545258, 3949598501, 970508564, 333838980, 1904445011, 3536602335, 3315495305, 3927833638, 544288708, 4170651832, 506226658, 675514483, 1128070245, 45496309, 1823907

loss: 17419.751953125


Step: 91 ___________________________________________
loss: 16692.427734375


Step: 92 ___________________________________________
loss: 10729.1201171875


Step: 93 ___________________________________________
loss: 24141.396484375


Step: 94 ___________________________________________
loss: 16821.310546875


Step: 95 ___________________________________________
loss: 20031.625


Step: 96 ___________________________________________
loss: 21111.234375


Step: 97 ___________________________________________
loss: 15394.5322265625


Step: 98 ___________________________________________
loss: 11158.6298828125


Step: 99 ___________________________________________
loss: 16789.201171875


final_seed: 1926411384


In [24]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 6807670.5000
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 6652792.0000
Epoch 3/10000
1/1 [==============================] - 0s 7ms/step - loss: 4700922.0000
Epoch 4/10000
1/1 [==============================] - 0s 8ms/step - loss: 209236.0469
Epoch 5/10000
1/1 [==============================] - 0s 9ms/step - loss: 137185.5469
Epoch 6/10000
1/1 [==============================] - 0s 5ms/step - loss: 160878.0781
Epoch 7/10000
1/1 [==============================] - 0s 8ms/step - loss: 72725.4297
Epoch 8/10000
1/1 [==============================] - 0s 9ms/step - loss: 55399.1367
Epoch 9/10000
1/1 [==============================] - 0s 5ms/step - loss: 65948.2422
Epoch 10/10000
1/1 [==============================] - 0s 8ms/step - loss: 49451.5000
Epoch 11/10000
1/1 [==============================] - 0s 5ms/step - loss: 60441.6562
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step 

1/1 [==============================] - 0s 4ms/step - loss: 48714.2305
Epoch 98/10000
1/1 [==============================] - 0s 5ms/step - loss: 58869.8867
Epoch 99/10000
1/1 [==============================] - 0s 5ms/step - loss: 55507.5508
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 73640.5078
Epoch 101/10000
1/1 [==============================] - 0s 5ms/step - loss: 27698.5723
Epoch 102/10000
1/1 [==============================] - 0s 5ms/step - loss: 60587.1602
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 76180.8984
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 72403.2031
Epoch 105/10000
1/1 [==============================] - 0s 5ms/step - loss: 69905.6641
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 37876.3125
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 43916.9961
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 36840.4727
Epoch 193/10000
1/1 [==============================] - 0s 5ms/step - loss: 61668.1211
Epoch 194/10000
1/1 [==============================] - 0s 5ms/step - loss: 65226.3906
Epoch 195/10000
1/1 [==============================] - 0s 5ms/step - loss: 42879.8633
Epoch 196/10000
1/1 [==============================] - 0s 5ms/step - loss: 27615.2715
Epoch 197/10000
1/1 [==============================] - 0s 5ms/step - loss: 57423.8281
Epoch 198/10000
1/1 [==============================] - 0s 6ms/step - loss: 73237.4922
Epoch 199/10000
1/1 [==============================] - 0s 5ms/step - loss: 61044.1680
Epoch 200/10000
1/1 [==============================] - 0s 6ms/step - loss: 71382.5859
Epoch 201/10000
1/1 [==============================] - 0s 5ms/step - loss: 42845.8008
Epoch 202/10000
1/1 [==============================] - 0s 6ms/step - loss: 44953.3008
Epoch 203/10000
1/1 [==============================] - 0s 7ms/step - l

1/1 [==============================] - 0s 6ms/step - loss: 53423.8867
Epoch 288/10000
1/1 [==============================] - 0s 7ms/step - loss: 55190.8906
Epoch 289/10000
1/1 [==============================] - 0s 6ms/step - loss: 39095.9258
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 58101.2305
Epoch 291/10000
1/1 [==============================] - 0s 6ms/step - loss: 59041.1367
Epoch 292/10000
1/1 [==============================] - 0s 6ms/step - loss: 60984.0195
Epoch 293/10000
1/1 [==============================] - 0s 7ms/step - loss: 69525.4297
Epoch 294/10000
1/1 [==============================] - 0s 5ms/step - loss: 60046.1367
Epoch 295/10000
1/1 [==============================] - 0s 6ms/step - loss: 21745.2793
Epoch 296/10000
1/1 [==============================] - 0s 6ms/step - loss: 39127.2227
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 31788.5703
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 49784.5586
Epoch 383/10000
1/1 [==============================] - 0s 5ms/step - loss: 40698.3867
Epoch 384/10000
1/1 [==============================] - 0s 5ms/step - loss: 59254.0000
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 39145.0547
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 62715.0430
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 66558.1875
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 68576.0078
Epoch 389/10000
1/1 [==============================] - 0s 5ms/step - loss: 77228.9219
Epoch 390/10000
1/1 [==============================] - 0s 5ms/step - loss: 53951.1367
Epoch 391/10000
1/1 [==============================] - 0s 5ms/step - loss: 91928.2109
Epoch 392/10000
1/1 [==============================] - 0s 6ms/step - loss: 99781.0391
Epoch 393/10000
1/1 [==============================] - 0s 4ms/step - l

...

1/1 [==============================] - 0s 4ms/step - loss: 46913.0312
Epoch 3021/10000
1/1 [==============================] - 0s 5ms/step - loss: 33815.9023
Epoch 3022/10000
1/1 [==============================] - 0s 4ms/step - loss: 41843.6211
Epoch 3023/10000
1/1 [==============================] - 0s 5ms/step - loss: 73042.3516
Epoch 3024/10000
1/1 [==============================] - 0s 6ms/step - loss: 47585.7812
Epoch 3025/10000
1/1 [==============================] - 0s 5ms/step - loss: 63506.3125
Epoch 3026/10000
1/1 [==============================] - 0s 5ms/step - loss: 45905.4688
Epoch 3027/10000
1/1 [==============================] - 0s 5ms/step - loss: 52130.4492
Epoch 3028/10000
1/1 [==============================] - 0s 5ms/step - loss: 63092.0820
Epoch 3029/10000
1/1 [==============================] - 0s 5ms/step - loss: 57403.5781
Epoch 3030/10000
1/1 [==============================] - 0s 4ms/step - loss: 59395.6758
Epoch 3031/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 4ms/step - loss: 43526.2930
Epoch 3115/10000
1/1 [==============================] - 0s 5ms/step - loss: 61909.4805
Epoch 3116/10000
1/1 [==============================] - 0s 5ms/step - loss: 27167.0684
Epoch 3117/10000
1/1 [==============================] - 0s 4ms/step - loss: 16712.1328
Epoch 3118/10000
1/1 [==============================] - 0s 5ms/step - loss: 30445.1816
Epoch 3119/10000
1/1 [==============================] - 0s 5ms/step - loss: 37113.0508
Epoch 3120/10000
1/1 [==============================] - 0s 5ms/step - loss: 20108.1934
Epoch 3121/10000
1/1 [==============================] - 0s 5ms/step - loss: 32340.9531
Epoch 3122/10000
1/1 [==============================] - 0s 4ms/step - loss: 53746.0781
Epoch 3123/10000
1/1 [==============================] - 0s 4ms/step - loss: 31817.1152
Epoch 3124/10000
1/1 [==============================] - 0s 5ms/step - loss: 28624.5918
Epoch 3125/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 5ms/step - loss: 43301.6133
Epoch 3209/10000
1/1 [==============================] - 0s 4ms/step - loss: 46284.7188
Epoch 3210/10000
1/1 [==============================] - 0s 4ms/step - loss: 44012.7852
Epoch 3211/10000
1/1 [==============================] - 0s 5ms/step - loss: 36129.5430
Epoch 3212/10000
1/1 [==============================] - 0s 5ms/step - loss: 23697.9766
Epoch 3213/10000
1/1 [==============================] - 0s 4ms/step - loss: 33773.7852
Epoch 3214/10000
1/1 [==============================] - 0s 5ms/step - loss: 53152.4062
Epoch 3215/10000
1/1 [==============================] - 0s 5ms/step - loss: 23504.6582
Epoch 3216/10000
1/1 [==============================] - 0s 5ms/step - loss: 67241.1484
Epoch 3217/10000
1/1 [==============================] - 0s 5ms/step - loss: 61145.2773
Epoch 3218/10000
1/1 [==============================] - 0s 4ms/step - loss: 63232.1523
Epoch 3219/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 4ms/step - loss: 55685.1289
Epoch 3303/10000
1/1 [==============================] - 0s 4ms/step - loss: 48965.9570
Epoch 3304/10000
1/1 [==============================] - 0s 4ms/step - loss: 54393.8398
Epoch 3305/10000
1/1 [==============================] - 0s 4ms/step - loss: 45927.6562
Epoch 3306/10000
1/1 [==============================] - 0s 4ms/step - loss: 40584.4570
Epoch 3307/10000
1/1 [==============================] - 0s 5ms/step - loss: 45614.2969
Epoch 3308/10000
1/1 [==============================] - 0s 4ms/step - loss: 19461.3828
Epoch 3309/10000
1/1 [==============================] - 0s 4ms/step - loss: 46098.1133
Epoch 3310/10000
1/1 [==============================] - 0s 5ms/step - loss: 57732.0625
Epoch 3311/10000
1/1 [==============================] - 0s 4ms/step - loss: 65292.1094
Epoch 3312/10000
1/1 [==============================] - 0s 4ms/step - loss: 47992.3125
Epoch 3313/10000
1/1 [==============================] - 0s 5

1/1 [==============================] - 0s 5ms/step - loss: 69997.0391
Epoch 3397/10000
1/1 [==============================] - 0s 5ms/step - loss: 28297.8301
Epoch 3398/10000
1/1 [==============================] - 0s 4ms/step - loss: 45771.1367
Epoch 3399/10000
1/1 [==============================] - 0s 4ms/step - loss: 46522.3594
Epoch 3400/10000
1/1 [==============================] - 0s 4ms/step - loss: 54414.8477
Epoch 3401/10000
1/1 [==============================] - 0s 5ms/step - loss: 27520.9922
Epoch 3402/10000
1/1 [==============================] - 0s 5ms/step - loss: 27770.9453
Epoch 3403/10000
1/1 [==============================] - 0s 5ms/step - loss: 54552.7812
Epoch 3404/10000
1/1 [==============================] - 0s 5ms/step - loss: 26273.2090
Epoch 3405/10000
1/1 [==============================] - 0s 4ms/step - loss: 57358.3125
Epoch 3406/10000
1/1 [==============================] - 0s 5ms/step - loss: 36816.5273
Epoch 3407/10000
1/1 [==============================] - 0s 6

In [25]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 591ms/step
[2782.135](test_target) - [[2405.4883]](prediction) = 376.6467187500002


In [26]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.13538046095893988

In [27]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [28]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.258939,-1.397925,1.268852,-0.94647,-1.118690,-0.551011,-0.770233,-1.233828,-0.425199,1.148205,...,1.277766,1.308949,1.412346,-1.413223,-1.238512,-1.303278,0.658911,1.028795,1.089129,-1.223704
1,0.071523,0.884312,-0.093583,-0.43679,-0.189921,-0.852453,-0.642042,0.018373,1.380677,0.140893,...,-0.114016,-0.190804,-0.643250,0.752439,0.028014,0.176141,0.754232,0.325951,0.236684,-0.002080
2,1.187416,0.513613,-1.175269,1.38326,1.308611,1.403464,1.412276,1.215455,-0.955478,-1.289098,...,-1.163750,-1.118144,-0.769096,0.660784,1.210497,1.127138,-1.413142,-1.354746,-1.325813,1.225784


train_target:


,Goiás - Consumo de Cimento (t)
0,2335.717
1,2657.527
2,2901.707


test_input:


...

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-1.158228,-2.357097,-1.338521,0.495067,-0.996697,-1.327004,0.145995,1.141532,0.318701,1.108405,...,0.964765,-1.482779,-2.260447,0.946592,0.252129,0.101107,0.76294,-1.953922,-1.523095,-2.072592


test_target:


,Goiás - Consumo de Cimento (t)
12,2782.135


1/1 [==============================] - 1s 541ms/step
Error: 376.6467187500002




[2940.812744140625,
 2985.573974609375,
 2968.128173828125,
 2814.720703125,
 2450.29931640625,
 2195.109375,
 2253.054931640625,
 2270.617919921875,
 2265.968994140625,
 2405.48828125]

In [29]:
display(targets)
display(predictions)

[3061.533,
 3181.778,
 2861.8974285714285,
 2517.2907428571425,
 2193.5458285714285,
 2270.346,
 2282.903,
 2367.716,
 2438.585,
 2782.135]

[2940.812744140625,
 2985.573974609375,
 2968.128173828125,
 2814.720703125,
 2450.29931640625,
 2195.109375,
 2253.054931640625,
 2270.617919921875,
 2265.968994140625,
 2405.48828125]

In [30]:
mae = mean_absolute_error(predictions, targets)
mae

172.878397265625

In [31]:
porcentage = mae/np.mean(targets)
porcentage

0.06659996743383378